In [ ]:
from gff_analyser import gffClasses, gffBuilder
from pybedtools import BedTool
import os
import sys
import pandas as pd

In [ ]:
#GTF_PATH = '/home/stud3/notebooks/homo_sapiens.104.mainChr.gtf'
GTF_PATH = '/mnt/workspace_stud/allstud/homo_sapiens.104.mainChr.gtf'

enhancer_bed = '/home/stud3/notebooks/ENCFF212UAV.bed'
blacklisted_bed = '/home/stud3/notebooks/ENCFF356LFX.bed'

frag = '/home/stud3/notebooks/stomach_SM-JF1O3_rep1_fragments_sorted.bed'

INPUT_PATH = '/mnt/workspace_stud/catlas_ref/cellXcCRE/'
H5AD = 'stomach_SM-JF1O3.h5ad' 
test = '/home/stud3/notebooks/stomach_more_cluster.h5ad'


In [ ]:
def generate_feature_files(fragments, threshold=10):
    
    object_list = gffBuilder.build_gff3_class(file=GTF_PATH)
    
    
    for element in object_list:
        features = element.count_features()

        element.generate_feature_gtf(gffdata_list=object_list, feature_keys=features)
        element.generate_promotor_gtf(gffdata_list=object_list, promotor_distance=2000)
        element.generate_tss_gtf(gffdata_list=object_list, tss_distance=100)
        element.generate_gene_body_gtf(gffdata_list=object_list)
    
        # Getting bedtools Path
        bedtools = os.path.join('/'.join(sys.executable.split('/')[:-1]),'bedtools')

        
        
        # Load the bed file into a pandas dataframe
        bed_df = pd.read_table(fragments, header=None, names=["chrom", "start", "end", "feature", "count", "strand"])

        # Filter the dataframe based on a count threshold
        
        filtered_df = bed_df[bed_df["count"] > threshold]

        filename = f"{fragments}.peak{threshold}.bed"
        # Print the filtered dataframe
        filtered_df.to_csv(filename ,sep='\t', index=None) 
        
        peakfile = f"{fragments}.peak{threshold}.bed"

        # Getting the peak-GTF
        intersect_cmd = f'{bedtools} intersect -a {GTF_PATH} -b {peakfile} > out/homo_sapiens.104.mainChr.peak{threshold}.gtf'
        os.system(intersect_cmd)
        print("Peaks done")

        # Getting the enhancer-GTF 
        intersect_cmd = f'{bedtools} intersect -a {GTF_PATH} -b {enhancer_bed} > out/homo_sapiens.104.mainChr.enhancer.gtf'
        os.system(intersect_cmd)
        print("Enhancers done")

        # Getting the exclusion_list-GTF
        intersect_cmd = f'{bedtools} intersect -a {GTF_PATH}  -b {blacklisted_bed} > out/homo_sapiens.104.mainChr.blacklisted.gtf'
        os.system(intersect_cmd)
        print("Blacklist done")

In [ ]:
generate_feature_files(fragments=frag)